In [1]:
from pyschedule import Scenario, solvers, plotters, alt

### Site Configuartion

Scanners

In [2]:
PET_Scanner = 1

Uptake Rooms

In [3]:
UptakeRooms_Count = 2

Hot Toilet

In [4]:
HotToilet_Count = 1

Workforce

In [5]:
Technologists_Count = 2
Clinical_Assistants_Count = 2
Admin_Count = 1

Operating Hours

In [6]:
Site_Operating_Hours_FlexMins = 2*60
Site_Operating_Standard_Mins = 12*60
Site_Operating_Mins = Site_Operating_Standard_Mins + Site_Operating_Hours_FlexMins

Admin_Operating_Mins = 12*60

Scan Limits

Estimated Scan Demand

In [7]:
PET30_Scan_Count = 2
PET60_Scan_Count = 1

In [8]:
import pulp
# Instantiate our problem class
model = pulp.LpProblem("Maximising Scans problem", pulp.LpMaximize)

Scan30 = pulp.LpVariable('Scan30', lowBound=0, cat='Integer')
Scan60 = pulp.LpVariable('Scan60', lowBound=0, cat='Integer')

# Objective function
model +=  Scan30 +  Scan60, "MaxScans"

# Constraints
model += 40 * Scan30 + 70 * Scan60 <= 780
model +=  Scan30 +  Scan60 <= 20
#model += Scan30 <= 1
model += Scan30 >= PET30_Scan_Count
model += Scan60 >= PET60_Scan_Count
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [9]:
if pulp.LpStatus[model.status]=='Optimal':
    print("Proceed Forward")
else:
    print("No Solution Exist")


Proceed Forward


### Create Scenario

In [10]:
# Load pyschedule and create a scenario with ten steps planning horizon
S = Scenario('PET_Schedule',horizon=2*Site_Operating_Mins)
S

###############################################

SCENARIO: PET_Schedule / horizon: 1680

OBJECTIVE: None

RESOURCES:

TASKS:

JOINT RESOURCES:

###############################################

### Create Resources

In [11]:
# Scanner resources
PET_Scanner = S.Resource('PET_Scanner',num=PET_Scanner)
S

###############################################

SCENARIO: PET_Schedule / horizon: 1680

OBJECTIVE: None

RESOURCES:
PET_Scanner

TASKS:

JOINT RESOURCES:

###############################################

In [12]:
UptakeRoom = S.Resource('UptakeRoom',num=UptakeRooms_Count)
S

###############################################

SCENARIO: PET_Schedule / horizon: 1680

OBJECTIVE: None

RESOURCES:
PET_Scanner
UptakeRoom

TASKS:

JOINT RESOURCES:

###############################################

In [13]:
# Workforce resources
Clinical_Assistant = S.Resource('Clinical_Assistant',num=Clinical_Assistants_Count)
Technologist = S.Resource('Technologist',num=Technologists_Count)
Admin = S.Resource('Admin',num=Admin_Count)
S

###############################################

SCENARIO: PET_Schedule / horizon: 1680

OBJECTIVE: None

RESOURCES:
PET_Scanner
UptakeRoom
Clinical_Assistant
Technologist
Admin

TASKS:

JOINT RESOURCES:

###############################################

### Create Tasks

In [14]:
for x in range(1,PET30_Scan_Count+1):
    print(x)
    globals()['D_%s' % x] = S.Task('D_%s' % x,length=5)
    globals()['BV_%s' % x] = S.Task('BV_%s' % x,length=2)
    globals()['C_%s' % x] = S.Task('C_%s' % x,length=5)
    globals()['SQ_%s' % x] = S.Task('SQ_%s' % x,length=10)
    globals()['SQV_%s' % x] = S.Task('SQV_%s' % x,length=4)
    globals()['I_%s' % x] = S.Task('I_%s' % x,length=5)
    globals()['PR_%s' % x] = S.Task('PR_%s' % x,length=50)
    globals()['S3_%s' % x] = S.Task('S3_%s' % x,length=30,delay_cost=1)
    
        
    globals()['D_%s' % x]  += Clinical_Assistant | Admin | Technologist
    globals()['BV_%s' % x] += Clinical_Assistant | Technologist 
    globals()['SQ_%s' % x] += Clinical_Assistant | Technologist
    globals()['C_%s' % x] += UptakeRoom, Clinical_Assistant | Technologist
    globals()['SQV_%s' % x] += UptakeRoom,  Technologist
    globals()['I_%s' % x] += UptakeRoom , Technologist
    globals()['PR_%s' % x] += UptakeRoom
    globals()['S3_%s' % x] += PET_Scanner, Technologist

    
    S+= globals()['D_%s' % x] < globals()['BV_%s' % x]
    S+= globals()['BV_%s' % x] < globals()['SQ_%s' % x]
    S+= globals()['SQ_%s' % x] < globals()['SQV_%s' % x]
    S+= globals()['SQV_%s' % x] < globals()['C_%s' % x]
    S+= globals()['C_%s' % x] < globals()['I_%s' % x]
    S+= globals()['I_%s' % x] < globals()['PR_%s' % x]
    S+= globals()['PR_%s' % x] < globals()['S3_%s' % x]
    S+= globals()['D_%s' % x] + 150 > globals()['S3_%s' % x]
    #S+= globals()['C_%s' % x] + 50 < globals()['S3_%s' % x]

1
2


In [15]:
S

###############################################

SCENARIO: PET_Schedule / horizon: 1680

OBJECTIVE: S3_1+S3_2

RESOURCES:
PET_Scanner
UptakeRoom
Clinical_Assistant
Technologist
Admin

TASKS:
D_1 : Clinical_Assistant|Admin|Technologist
BV_1 : Clinical_Assistant|Technologist
C_1 : UptakeRoom,Clinical_Assistant|Technologist
SQ_1 : Clinical_Assistant|Technologist
SQV_1 : UptakeRoom,Technologist
I_1 : UptakeRoom,Technologist
PR_1 : UptakeRoom
S3_1 : PET_Scanner,Technologist
D_2 : Clinical_Assistant|Admin|Technologist
BV_2 : Clinical_Assistant|Technologist
C_2 : UptakeRoom,Clinical_Assistant|Technologist
SQ_2 : Clinical_Assistant|Technologist
SQV_2 : UptakeRoom,Technologist
I_2 : UptakeRoom,Technologist
PR_2 : UptakeRoom
S3_2 : PET_Scanner,Technologist

JOINT RESOURCES:
Clinical_Assistant|Admin|Technologist : D_1
Clinical_Assistant|Technologist : BV_1
Clinical_Assistant|Technologist : C_1
Clinical_Assistant|Technologist : SQ_1
Clinical_Assistant|Admin|Technologist : D_2
Clinical_Assistant|Te

In [16]:
for x in range(PET30_Scan_Count+1,PET30_Scan_Count+PET60_Scan_Count+1):
    print(x)
    globals()['D_%s' % x] = S.Task('D_%s' % x,length=15)
    globals()['BV_%s' % x] = S.Task('BV_%s' % x,length=2)
    globals()['C_%s' % x] = S.Task('C_%s' % x,length=5)
    globals()['SQ_%s' % x] = S.Task('SQ_%s' % x,length=10)
    globals()['SQV_%s' % x] = S.Task('SQV_%s' % x,length=4)
    globals()['I_%s' % x] = S.Task('I_%s' % x,length=5)
    globals()['PR_%s' % x] = S.Task('PR_%s' % x,length=50)
    globals()['S6_%s' % x] = S.Task('S6_%s' % x,length=60,delay_cost=1)
    
    globals()['D_%s' % x]  += Clinical_Assistant | Admin | Technologist
    globals()['BV_%s' % x] += Clinical_Assistant | Technologist 
    globals()['SQ_%s' % x] += Clinical_Assistant | Technologist
    globals()['C_%s' % x] += UptakeRoom, Clinical_Assistant | Technologist
    globals()['SQV_%s' % x] += UptakeRoom, Clinical_Assistant | Technologist
    globals()['I_%s' % x] += UptakeRoom , Technologist
    globals()['PR_%s' % x] += UptakeRoom
    globals()['S6_%s' % x] += PET_Scanner, Technologist

    
    S+= globals()['D_%s' % x] < globals()['BV_%s' % x]
    S+= globals()['BV_%s' % x] < globals()['SQ_%s' % x]
    S+= globals()['SQ_%s' % x] < globals()['SQV_%s' % x]
    S+= globals()['SQV_%s' % x] < globals()['C_%s' % x]
    S+= globals()['C_%s' % x] < globals()['I_%s' % x]
    S+= globals()['I_%s' % x] < globals()['PR_%s' % x]
    S+= globals()['PR_%s' % x] < globals()['S6_%s' % x]
    S+= globals()['D_%s' % x] + 150 > globals()['S6_%s' % x]
    #S+= globals()['C_%s' % x] + 70 < globals()['S6_%s' % x]

3


In [17]:
print(S)

###############################################

SCENARIO: PET_Schedule / horizon: 1680

OBJECTIVE: S3_1+S3_2+S6_3

RESOURCES:
PET_Scanner
UptakeRoom
Clinical_Assistant
Technologist
Admin

TASKS:
D_1 : Clinical_Assistant|Admin|Technologist
BV_1 : Clinical_Assistant|Technologist
C_1 : UptakeRoom,Clinical_Assistant|Technologist
SQ_1 : Clinical_Assistant|Technologist
SQV_1 : UptakeRoom,Technologist
I_1 : UptakeRoom,Technologist
PR_1 : UptakeRoom
S3_1 : PET_Scanner,Technologist
D_2 : Clinical_Assistant|Admin|Technologist
BV_2 : Clinical_Assistant|Technologist
C_2 : UptakeRoom,Clinical_Assistant|Technologist
SQ_2 : Clinical_Assistant|Technologist
SQV_2 : UptakeRoom,Technologist
I_2 : UptakeRoom,Technologist
PR_2 : UptakeRoom
S3_2 : PET_Scanner,Technologist
D_3 : Clinical_Assistant|Admin|Technologist
BV_3 : Clinical_Assistant|Technologist
C_3 : UptakeRoom,Clinical_Assistant|Technologist
SQ_3 : Clinical_Assistant|Technologist
SQV_3 : UptakeRoom,Clinical_Assistant|Technologist
I_3 : UptakeRoom

Resource Requirement

In [ ]:
%%time
# compute and print a schedule
solvers.mip.solve(S,time_limit=10,random_seed=42,msg=1)
print(S.solution())

In [ ]:
%matplotlib inline
plotters.matplotlib.plot(S,img_filename='op5.png')

In [ ]:
@@
if solvers.mip.solve(S,msg=1):
    %matplotlib inline
    plotters.matplotlib.plot(S,fig_size=(12,5))
else:
    print('no solution found')

In [ ]:
pulp.pulpTestAll()